# Flash Evaluation on Unicorn Dataset:

This notebook is dedicated to evaluating Flash on the Unicorn datasets, which are graph-level in nature. We employ Flash in graph-level detection mode to analyze this dataset effectively. Upon completion of the notebook execution, the results will be presented.

## Dataset Access: 
- The Unicorn dataset can be accessed at the following link: [Unicorn Dataset](https://github.com/margoseltzer/shellshock-apt).

## Data Parsing and Execution:
- The script automatically downloads the parsed dataset.
- To obtain the evaluation results, execute all cells within this notebook.

## Model Training and Execution Flexibility:
- By default, the notebook operates using pre-trained model weights.
- Additionally, this notebook offers the flexibility to set parameters for training Graph Neural Networks (GNNs) and word2vec models from scratch.
- You can then utilize these freshly trained models to conduct the evaluation. 

Follow these guidelines for a thorough and efficient analysis of the Unicorn datasets using Flash.


In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import os
import torch.nn.functional as F
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import multiprocessing

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline

In [38]:
import gdown
url = "https://drive.google.com/file/d/1dmezgT9zQ-8ydHrXvJmxwJN-LjPtumbU/view"
gdown.download(url, quiet=False, use_cookies=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1dmezgT9zQ-8ydHrXvJmxwJN-LjPtumbU
From (redirected): https://drive.google.com/uc?id=1dmezgT9zQ-8ydHrXvJmxwJN-LjPtumbU&confirm=t&uuid=59a182f2-679b-4eb2-ae70-835020c9108a
To: /home/tpiuser2/prov_project/amaan_flash/Flash-IDS-main/unicorn.zip
100%|██████████| 97.6M/97.6M [00:06<00:00, 15.9MB/s]


'unicorn.zip'

In [39]:
import zipfile

def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

zip_path = 'unicorn.zip'
extract_to = 'unicorn'
unzip_file(zip_path, extract_to)

In [40]:
Train_Gnn = True
Train_Word2vec = True

In [41]:
from pprint import pprint
import gzip
from sklearn.manifold import TSNE
import json
import copy
import os

In [42]:
import os.path as osp
import csv
def show(str):
	print (str + ' ' + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time())))

def parse_data():
    for i in range(3):
        os.system('tar -zxvf camflow-attack-' + str(i) + '.gz.tar')
    for i in range(13):
        os.system('tar -zxvf camflow-benign-' + str(i) + '.gz.tar')

    os.system('rm error.log')
    os.system('rm parse-error-camflow-*')
    show('Start processing.')
    for i in range(25):
        show('Attack graph ' + str(i+125))
        f = open('camflow-attack.txt.'+str(i), 'r')
        fw = open('unicorn/'+str(i+125)+'.txt', 'w')
        for line in f:
                tempp = line.strip('\n').split('\t')
                temp = []
                temp.append(tempp[0])
                temp.append(tempp[2].split(':')[0])
                temp.append(tempp[1])
                temp.append(tempp[2].split(':')[1])
                temp.append(tempp[2].split(':')[2])
                temp.append(tempp[2].split(':')[3])
                fw.write(temp[0]+'\t'+temp[1]+'\t'+temp[2]+'\t'+temp[3]+'\t'+temp[4]+'\t'+temp[5]+'\n')
        f.close()
        fw.close()
        os.system('rm camflow-attack.txt.' + str(i))

    for i in range(125):
        show('Benign graph ' + str(i))
        f = open('camflow-normal.txt.'+str(i), 'r')
        fw = open('unicorn/'+str(i)+'.txt', 'w')
        for line in f:
                tempp = line.strip('\n').split('\t')
                temp = []
                temp.append(tempp[0])
                temp.append(tempp[2].split(':')[0])
                temp.append(tempp[1])
                temp.append(tempp[2].split(':')[1])
                temp.append(tempp[2].split(':')[2])
                temp.append(tempp[2].split(':')[3])
                fw.write(temp[0]+'\t'+temp[1]+'\t'+temp[2]+'\t'+temp[3]+'\t'+temp[4]+'\t'+temp[5]+'\n')
        f.close()
        fw.close()
        os.system('rm camflow-normal.txt.' + str(i))
    show('Done.')

In [43]:
def prepare_graph(df):
    def process_node(node, action, node_dict, label_dict, dummies, node_type):
        node_dict.setdefault(node, []).append(action)
        label_dict[node] = dummies.get(getattr(row, node_type), -1)  

    nodes = {}
    labels = {}
    edges = []
    dummies = {
        "7998762093665332071": 0, "14709879154498484854": 1, "10991425273196493354": 2,
        "14871526952859113360": 3, "8771628573506871447": 4, "7877121489144997480": 5,
        "17841021884467483934": 6, "7895447931126725167": 7, "15125250455093594050": 8,
        "8664433583651064836": 9, "14377490526132269506": 10, "15554536683409451879": 11,
        "8204541918505434145": 12, "14356114695140920775": 13
    }

    for row in df.itertuples():
        process_node(row.actorID, row.action, nodes, labels, dummies, 'actor_type')
        process_node(row.objectID, row.action, nodes, labels, dummies, 'object')

        edges.append((row.actorID, row.objectID))

    features = [nodes[node] for node in nodes]
    feat_labels = [labels[node] for node in nodes]
    
    
    edge_index = [[], []]
    # for src, dst in edges:
    #     src_index = list(nodes.keys()).index(src)
    #     dst_index = list(nodes.keys()).index(dst)
    #     edge_index[0].append(src_index)
    #     edge_index[1].append(dst_index)
    
    node_index_map = {node: i for i, node in enumerate(nodes.keys())}
    for src, dst in tqdm(edges):
        src_index = node_index_map[src]
        dst_index = node_index_map[dst]
        edge_index[0].append(src_index)
        edge_index[1].append(dst_index)

    return features, feat_labels, edge_index, list(nodes.keys())


In [44]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv, GATConv
import torch.nn.functional as F
import torch.nn as nn

class GCN(torch.nn.Module):
    def __init__(self,in_channel,out_channel):
        super().__init__()
        self.conv1 = SAGEConv(in_channel, 32, normalize=True)
        self.conv2 = SAGEConv(32, 20, normalize=True)
        self.linear = nn.Linear(in_features=20,out_features=out_channel)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        x = self.linear(x)
        return F.softmax(x, dim=1)

In [45]:
from gensim.models.callbacks import CallbackAny2Vec
import gensim
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm
import time

class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('trained_weights/unicorn/unicorn.model')
        self.epoch += 1

In [46]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [47]:
logger = EpochLogger()
saver = EpochSaver()

In [48]:
if Train_Word2vec:
    comb_data = []
    for i in range(20):
        print(i)
        f = open(f"unicorn/{i}.txt")
        data = f.read().split('\n')
        data = [line.split('\t') for line in data]
        comb_data = comb_data + data

    df = pd.DataFrame (comb_data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()
    phrases,labels,edges,mapp = prepare_graph(df)
    
    word2vec = Word2Vec(sentences=phrases, vector_size=30, window=5, min_count=1, workers=8,epochs=300,callbacks=[saver,logger])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


100%|██████████| 1507701/1507701 [00:01<00:00, 1188298.07it/s]


Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #20 start
Epoch #20 end
Epoch #21 start
Epoch #21 end
Epoch #22 start
Epoch #22 end
Epoch #23 start
Epoch #23 end
Epoch #24 start
Epoch #24 end
Epoch #25 start
Epoch #25 end
Epoch #26 start
Epoch #26 end
Epoch #27 start
Epoch #27 end
Epoch #28 start
Epoch #28 end
Epoch #29 start
Epoch #29 end
Epoch #30 start
Epoch #30 end
Epoch #31 start
Epoch #31 end
Epoch #32 start
Epoch #32 end
Epoch #33 start
Epoch #33 end


In [49]:
from sklearn.utils import class_weight
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GCN(30,14).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [50]:
from collections import Counter
import math

class PositionalEncoder:

    def __init__(self, d_model, max_len=100000):
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        self.pe = torch.zeros(max_len, d_model)
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)

    def embed(self, x):
        return x + self.pe[:x.size(0)]

def infer(document):
    
    if Train_Word2vec:                  ## added.
        word_embeddings = [word2vec.wv[word] for word in document if word in  word2vec.wv]
    else:
        word_embeddings = [w2vmodel.wv[word] for word in document if word in  w2vmodel.wv]
        
    if not word_embeddings:
        return np.zeros(20)

    output_embedding = torch.tensor(word_embeddings, dtype=torch.float)
    if len(document) < 100000:
        output_embedding = encoder.embed(output_embedding)

    output_embedding = output_embedding.detach().cpu().numpy()
    return np.mean(output_embedding, axis=0)

encoder = PositionalEncoder(30)
w2vmodel = Word2Vec.load("trained_weights/unicorn/unicorn.model")

In [51]:
from torch_geometric import utils

################################## Training Main Model #####################################
if Train_Gnn:
    for i in range(95):
        f = open(f"unicorn/{i}.txt")
        data = f.read().split('\n')

        data = [line.split('\t') for line in data]
        df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
        df.sort_values(by='timestamp', ascending=True,inplace=True)
        df = df.dropna()
        phrases,labels,edges,mapp = prepare_graph(df)

        criterion = CrossEntropyLoss()

        nodes = [infer(x) for x in phrases]
        nodes = np.array(nodes)  

        graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
        graph.n_id = torch.arange(graph.num_nodes)
        mask = torch.tensor([True]*graph.num_nodes, dtype=torch.bool)

        for m_n in range(20):
            loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
            total_loss = 0
            for subg in loader:
                model.train()
                optimizer.zero_grad() 
                out = model(subg.x, subg.edge_index) 
                loss = criterion(out, subg.y) 
                loss.backward() 
                optimizer.step()      
                total_loss += loss.item() * subg.batch_size

            loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
            for subg in loader:
              model.eval()
              out = model(subg.x, subg.edge_index)
              sorted, indices = out.sort(dim=1,descending=True)
              conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
              conf = (conf - conf.min()) / conf.max()
              pred = indices[:,0]
              cond = (pred == subg.y)
              mask[subg.n_id[cond]] = False

            print(f'Model# {m_n}. {mask.sum().item()} nodes still misclassified \n')
            torch.save(model.state_dict(), f'trained_weights/unicorn/unicorn{m_n}.pth')

100%|██████████| 116685/116685 [00:00<00:00, 648357.11it/s]


Model# 0. 19594 nodes still misclassified 

Model# 1. 16067 nodes still misclassified 

Model# 2. 12690 nodes still misclassified 

Model# 3. 8135 nodes still misclassified 

Model# 4. 6142 nodes still misclassified 

Model# 5. 4936 nodes still misclassified 

Model# 6. 4693 nodes still misclassified 

Model# 7. 4460 nodes still misclassified 

Model# 8. 4338 nodes still misclassified 

Model# 9. 4178 nodes still misclassified 

Model# 10. 4101 nodes still misclassified 

Model# 11. 3503 nodes still misclassified 

Model# 12. 3472 nodes still misclassified 

Model# 13. 3416 nodes still misclassified 

Model# 14. 3376 nodes still misclassified 

Model# 15. 3329 nodes still misclassified 

Model# 16. 3295 nodes still misclassified 

Model# 17. 2770 nodes still misclassified 

Model# 18. 2753 nodes still misclassified 

Model# 19. 2750 nodes still misclassified 



100%|██████████| 97195/97195 [00:00<00:00, 1284847.40it/s]


Model# 0. 5945 nodes still misclassified 

Model# 1. 5245 nodes still misclassified 

Model# 2. 4104 nodes still misclassified 

Model# 3. 3847 nodes still misclassified 

Model# 4. 3284 nodes still misclassified 

Model# 5. 3047 nodes still misclassified 

Model# 6. 2982 nodes still misclassified 

Model# 7. 2916 nodes still misclassified 

Model# 8. 2190 nodes still misclassified 

Model# 9. 1939 nodes still misclassified 

Model# 10. 1908 nodes still misclassified 

Model# 11. 1619 nodes still misclassified 

Model# 12. 1610 nodes still misclassified 

Model# 13. 1599 nodes still misclassified 

Model# 14. 1596 nodes still misclassified 

Model# 15. 1591 nodes still misclassified 

Model# 16. 1585 nodes still misclassified 

Model# 17. 1578 nodes still misclassified 

Model# 18. 1570 nodes still misclassified 

Model# 19. 1557 nodes still misclassified 



100%|██████████| 87587/87587 [00:00<00:00, 1294907.33it/s]


...

Model# 0. 1038 nodes still misclassified 

Model# 1. 1018 nodes still misclassified 

Model# 2. 1004 nodes still misclassified 

Model# 3. 1001 nodes still misclassified 

Model# 4. 1001 nodes still misclassified 

Model# 5. 997 nodes still misclassified 

Model# 6. 635 nodes still misclassified 

Model# 7. 630 nodes still misclassified 

Model# 8. 626 nodes still misclassified 

Model# 9. 625 nodes still misclassified 

Model# 10. 622 nodes still misclassified 

Model# 11. 482 nodes still misclassified 

Model# 12. 425 nodes still misclassified 

Model# 13. 411 nodes still misclassified 

Model# 14. 395 nodes still misclassified 

Model# 15. 393 nodes still misclassified 

Model# 16. 351 nodes still misclassified 

Model# 17. 343 nodes still misclassified 

Model# 18. 309 nodes still misclassified 

Model# 19. 309 nodes still misclassified 



100%|██████████| 48717/48717 [00:00<00:00, 1480305.05it/s]


Model# 0. 779 nodes still misclassified 

Model# 1. 760 nodes still misclassified 

Model# 2. 737 nodes still misclassified 

Model# 3. 671 nodes still misclassified 

Model# 4. 661 nodes still misclassified 

Model# 5. 648 nodes still misclassified 

Model# 6. 516 nodes still misclassified 

Model# 7. 386 nodes still misclassified 

Model# 8. 374 nodes still misclassified 

Model# 9. 361 nodes still misclassified 

Model# 10. 348 nodes still misclassified 

Model# 11. 340 nodes still misclassified 

Model# 12. 338 nodes still misclassified 

Model# 13. 336 nodes still misclassified 

Model# 14. 334 nodes still misclassified 

Model# 15. 310 nodes still misclassified 

Model# 16. 294 nodes still misclassified 

Model# 17. 292 nodes still misclassified 

Model# 18. 292 nodes still misclassified 

Model# 19. 292 nodes still misclassified 



100%|██████████| 48673/48673 [00:00<00:00, 1424350.85it/s]


Model# 0. 554 nodes still misclassified 

Model# 1. 539 nodes still misclassified 

Model# 2. 472 nodes still misclassified 

Model# 3. 466 nodes still misclassified 

Model# 4. 422 nodes still misclassified 

Model# 5. 372 nodes still misclassified 

Model# 6. 371 nodes still misclassified 

Model# 7. 367 nodes still misclassified 

Model# 8. 366 nodes still misclassified 

Model# 9. 362 nodes still misclassified 

Model# 10. 359 nodes still misclassified 

Model# 11. 358 nodes still misclassified 

Model# 12. 357 nodes still misclassified 

Model# 13. 356 nodes still misclassified 

Model# 14. 356 nodes still misclassified 

Model# 15. 355 nodes still misclassified 

Model# 16. 355 nodes still misclassified 

Model# 17. 355 nodes still misclassified 

Model# 18. 355 nodes still misclassified 

Model# 19. 278 nodes still misclassified 



### Validation

In [52]:
for i in range(95,98):
    print(f"Graph #: {i}")
    f = open(f"unicorn/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()

    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)  

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool)

    for m_n in range(20):
        model.load_state_dict(torch.load(f'trained_weights/unicorn/unicorn{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], torch.tensor([False]*len(flag[graph.n_id[cond]]), dtype=torch.bool))
            
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100)

Graph #: 95


100%|██████████| 48518/48518 [00:00<00:00, 1353341.41it/s]


397 1.8583532275429482
Graph #: 96


100%|██████████| 48642/48642 [00:00<00:00, 1442648.39it/s]


346 2.1005342399222924
Graph #: 97


100%|██████████| 48671/48671 [00:00<00:00, 1116549.46it/s]


347 2.0265140454359636


### Testing

In [53]:
thresh = 330

In [54]:
correct_benign = 0

for i in range(100,125):
    print(f"Graph #: {i}")
    f = open(f"unicorn/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()

    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)  

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool)

    for m_n in range(20):
        model.load_state_dict(torch.load(f'trained_weights/unicorn/unicorn{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], torch.tensor([False]*len(flag[graph.n_id[cond]]), dtype=torch.bool))

    if flag.sum().item() <= thresh:
        correct_benign = correct_benign + 1
            
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100)

Graph #: 100


100%|██████████| 48728/48728 [00:00<00:00, 1373254.17it/s]


211 1.3069871159563924
Graph #: 101


100%|██████████| 48577/48577 [00:00<00:00, 1340968.18it/s]


406 2.0751341681574242
Graph #: 102


...

290 1.7777232881750749
Graph #: 123


100%|██████████| 38934/38934 [00:00<00:00, 1434856.62it/s]


347 2.656561016689634
Graph #: 124


100%|██████████| 38814/38814 [00:00<00:00, 1450675.58it/s]


378 2.2198731501057085


In [55]:
correct_attack = 0

for i in range(125,150):
    print(f"Graph #: {i}")
    f = open(f"unicorn/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()
    
    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)  
    
    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool)

    for m_n in range(20):
        model.load_state_dict(torch.load(f'trained_weights/unicorn/unicorn{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], torch.tensor([False]*len(flag[graph.n_id[cond]]), dtype=torch.bool))

    if  flag.sum().item() > thresh:
        correct_attack = correct_attack + 1
   
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100)

Graph #: 125


100%|██████████| 38910/38910 [00:00<00:00, 1349867.81it/s]


306 2.8042521994134897
Graph #: 126


100%|██████████| 38921/38921 [00:00<00:00, 772552.42it/s]


280 2.083333333333333
Graph #: 127


...

420 2.8345818991698724
Graph #: 148


100%|██████████| 38799/38799 [00:00<00:00, 1383599.32it/s]


354 2.311610291236777
Graph #: 149


100%|██████████| 38807/38807 [00:00<00:00, 1378831.96it/s]


264 1.65860400829302


In [56]:
TP = correct_attack
FP = 25 - correct_benign
TN = correct_benign
FN = 25 - correct_attack

FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
TPR = TP / (TP + FN) if (TP + FN) > 0 else 0

print(f"Number of True Positives (TP): {TP}")
print(f"Number of False Positives (FP): {FP}")
print(f"Number of False Negatives (FN): {FN}")
print(f"Number of True Negatives (TN): {TN}\n")

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TPR  
print(f"Precision: {precision}")
print(f"Recall: {recall}")

fscore = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f"Fscore: {fscore}\n")

Number of True Positives (TP): 13
Number of False Positives (FP): 11
Number of False Negatives (FN): 12
Number of True Negatives (TN): 14

Precision: 0.5416666666666666
Recall: 0.52
Fscore: 0.5306122448979592

